In [259]:
#import dependencies

import pandas as pd

In [260]:
#read in csvs

electiondata = pd.read_csv("1976-2016-president.csv")
rate1 = pd.read_csv("1980-2014 November General Election - Turnout Rates.csv")
rate2 = pd.read_csv("2016 November General Election - Turnout Rates.csv")
electoralvotes = pd.read_csv("Available Electoral votes .csv")

In [261]:
####pull party info for normalization


#parties = electiondata[["year","state","candidate", "party"]]
#parties.drop_duplicates(inplace=True)


#parties.to_csv("allparties.csv", index=False)
#parties

In [262]:
#clean up voting data 

#file one
rate1.rename(columns={"Unnamed: 0": "year", "Denominators" : "Eligible_voting_pop"}, inplace= True)
voting_data_1 = rate1[["year", "State", "Eligible_voting_pop"]]
voting_data_1 = voting_data_1.drop(voting_data_1.index[0])

#file two
rate2.rename(columns={"Denominators" : "Eligible_voting_pop"}, inplace= True)
rate2["year"] = '2016'
voting_data_2 = rate2[["year", "State", "Eligible_voting_pop"]]
voting_data_2 = voting_data_2.drop(voting_data_2.index[0])

In [263]:
#merge dataframes

total_voting = voting_data_2.append(voting_data_1)

In [264]:
#check dtypes on dataframes

#total_voting.dtypes
    #year is object

#electiondata.dtypes
    #year is int64

In [265]:
#convert year to int for merge

total_voting['year'] = total_voting.year.astype(str).astype(int)

In [266]:
#merge dataframes 

merged_df = electiondata.merge(total_voting, how='left', left_on=['state','year'], right_on=["State","year"])


In [267]:
#merge electoral college votes dataframe and drop columns

working_df = merged_df.merge(electoralvotes, how='left', left_on=['state','year'], right_on=["State","Year"])
final_working_df = working_df.drop(['state_po', 'state_fips', "state_cen", "state_ic", "office", "version", "notes", "State ID", "State_x","State_y", "Year", "writein"], axis=1)


In [268]:
#create key for merge and man
final_working_df["new"] = final_working_df["candidate"] + final_working_df["year"].astype(str)

final_working_df.replace("Mitt, Romney", "Romney, Mitt", inplace=True)
final_working_df.replace("Hedges, Jim", "Hedges, James", inplace=True)
final_working_df.replace("Haylard, Helen", "Halyard, Helen", inplace=True)
final_working_df.replace("Stevens, Thomas Robert ""Tom""", "Stevens, Thomas R.", inplace=True)
final_working_df.replace("White, Jerry", "White, Jerome ""Jerry""", inplace=True)


final_working_df["candidate"] = final_working_df['candidate'].fillna("unknown")
final_working_df["party"] = final_working_df['party'].fillna("unknown")
final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].fillna(.00000000000000000000000000000000000000000000000001)

final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].replace({'\$': '', ',': ''}, regex=True)
final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].astype(str).astype(float).round(0)

In [269]:
#read in manipulated party csv data

allparties = pd.read_csv("allparties.csv")

#merge all parties with final working for normalized party info 
final_working_df = final_working_df.merge(allparties,how= "left", left_on= ["candidate", "state", "year"], right_on=["candidate", "state", "year"])


In [270]:
###TROUBLESHOOT CELL IGNORE!!!!

#final_working_df = final_working_df.loc[final_working_df["year"] == 2000]
#final_working_df = final_working_df.loc[final_working_df["state"] == "Oregon"]
#final_working_df = final_working_df.loc[final_working_df["candidate"] == 'Other']

#final_working_df

In [271]:
# fill join column for search and replace

final_working_df["party_y"] = final_working_df['party_y'].fillna(0)

for row in final_working_df["party_y"]:
    final_working_df.loc[final_working_df["party_y"] != 0, ["party_x"]] = final_working_df["party_y"]

In [272]:
#rename and drop

final_working_df.rename(columns={"party_x" : "party"}, inplace=True)
final_working_df = final_working_df.drop("party_y", axis= 1)
final_working_df = final_working_df.drop_duplicates()

In [273]:
final_working_df = final_working_df.groupby(["year", "state", "candidate", "party", "totalvotes", "Eligible_voting_pop", "electoralvotes", "new"]).sum().reset_index()

In [274]:
###TROUBLESHOOT CELL IGNORE!!!!

#final_working_df = final_working_df.loc[final_working_df["year"] == 2016]
#final_working_df = final_working_df.loc[final_working_df["state"] == "Oregon"]
#final_working_df

In [275]:
#find top candidates by popular vote

find_top_candidates_df = final_working_df[["year","candidate", "candidatevotes"]]

find_top_candidates_df = find_top_candidates_df.groupby(["candidate", "year"]).sum().reset_index()

find_top_candidates_df= find_top_candidates_df.sort_values(by=["year", "candidatevotes"], ascending=False).groupby("year").head(2)

top_candidates_df = pd.DataFrame(find_top_candidates_df)

#create key for merge
top_candidates_df["new"] = top_candidates_df["candidate"] + top_candidates_df["year"].astype(str)

#create dataframe with only key for merge
#loop through to deduplicate 
only_winners = top_candidates_df[["new"]]

toppy = []
for row in only_winners["new"]:
    if row not in toppy:
        toppy.append(row)

toppy=pd.DataFrame(toppy)

In [276]:
#merge to create column for flag

final_working_df = final_working_df.merge(toppy, how="left", left_on= "new", right_on= 0)

#fill na in new columnt with 0
final_working_df[0] = final_working_df[0].fillna(0)


final_working_df

,year,state,candidate,party,totalvotes,Eligible_voting_pop,electoralvotes,new,candidatevotes,0
0,1976,Alabama,"Bubar, Benjamin """"Ben""""",prohibition,1182850,0.0,9.0,"Bubar, Benjamin """"Ben""""1976",6669,0
1,1976,Alabama,"Carter, Jimmy",democrat,1182850,0.0,9.0,"Carter, Jimmy1976",659170,"Carter, Jimmy1976"
2,1976,Alabama,"Ford, Gerald",republican,1182850,0.0,9.0,"Ford, Gerald1976",504070,"Ford, Gerald1976"
3,1976,Alabama,"Hall, Gus",communist party use,1182850,0.0,9.0,"Hall, Gus1976",1954,0
4,1976,Alabama,"Macbride, Roger",libertarian,1182850,0.0,9.0,"Macbride, Roger1976",1481,0
...,...,...,...,...,...,...,...,...,...,...
3361,2016,Wyoming,"Castle, Darrell L.",constitution party,258788,429682.0,3.0,"Castle, Darrell L.2016",2042,0
3362,2016,Wyoming,"Clinton, Hillary",democrat,258788,429682.0,3.0,"Clinton, Hillary2016",55973,"Clinton, Hillary2016"
3363,2016,Wyoming,"Johnson, Gary",libertarian,258788,429682.0,3.0,"Johnson, Gary2016",13287,0
3364,2016,Wyoming,Over Vote,unknown,258788,429682.0,3.0,Over Vote2016,278,0


In [277]:
#using flag, change candidate name to other if not in top 2

final_working_df.loc[final_working_df[0] == 0, ["candidate"]] = 'Other'

#drop flag column
final_working_df = final_working_df.drop([0], axis=1)

final_working_df

,year,state,candidate,party,totalvotes,Eligible_voting_pop,electoralvotes,new,candidatevotes
0,1976,Alabama,Other,prohibition,1182850,0.0,9.0,"Bubar, Benjamin """"Ben""""1976",6669
1,1976,Alabama,"Carter, Jimmy",democrat,1182850,0.0,9.0,"Carter, Jimmy1976",659170
2,1976,Alabama,"Ford, Gerald",republican,1182850,0.0,9.0,"Ford, Gerald1976",504070
3,1976,Alabama,Other,communist party use,1182850,0.0,9.0,"Hall, Gus1976",1954
4,1976,Alabama,Other,libertarian,1182850,0.0,9.0,"Macbride, Roger1976",1481
...,...,...,...,...,...,...,...,...,...
3361,2016,Wyoming,Other,constitution party,258788,429682.0,3.0,"Castle, Darrell L.2016",2042
3362,2016,Wyoming,"Clinton, Hillary",democrat,258788,429682.0,3.0,"Clinton, Hillary2016",55973
3363,2016,Wyoming,Other,libertarian,258788,429682.0,3.0,"Johnson, Gary2016",13287
3364,2016,Wyoming,Other,unknown,258788,429682.0,3.0,Over Vote2016,278


In [278]:
find_top_candidates_state_df = final_working_df[["year","state","candidate", "candidatevotes", "new"]]

find_top_candidates_state_df

,year,state,candidate,candidatevotes,new
0,1976,Alabama,Other,6669,"Bubar, Benjamin """"Ben""""1976"
1,1976,Alabama,"Carter, Jimmy",659170,"Carter, Jimmy1976"
2,1976,Alabama,"Ford, Gerald",504070,"Ford, Gerald1976"
3,1976,Alabama,Other,1954,"Hall, Gus1976"
4,1976,Alabama,Other,1481,"Macbride, Roger1976"
...,...,...,...,...,...
3361,2016,Wyoming,Other,2042,"Castle, Darrell L.2016"
3362,2016,Wyoming,"Clinton, Hillary",55973,"Clinton, Hillary2016"
3363,2016,Wyoming,Other,13287,"Johnson, Gary2016"
3364,2016,Wyoming,Other,278,Over Vote2016


In [279]:
#sum candidate votes

find_top_candidates_state_df = find_top_candidates_state_df.groupby(["candidate", "year", "state"]).sum().reset_index()

#sort to get top candidate
find_top_candidates_state_df= find_top_candidates_state_df.sort_values(by=[ "year","candidatevotes", "state"], ascending=False)
find_top_candidates_state_df = find_top_candidates_state_df.sort_values(["year", "state"]).groupby(["state","year"]).head(1)

#create key for merge
find_top_candidates_state_df["new"] = find_top_candidates_state_df["candidate"] + find_top_candidates_state_df["year"].astype(str)

find_top_candidates_state_df

,candidate,year,state,candidatevotes,new
200,"Carter, Jimmy",1976,Alabama,659170,"Carter, Jimmy1976"
551,"Ford, Gerald",1976,Alaska,71555,"Ford, Gerald1976"
552,"Ford, Gerald",1976,Arizona,418642,"Ford, Gerald1976"
203,"Carter, Jimmy",1976,Arkansas,498604,"Carter, Jimmy1976"
554,"Ford, Gerald",1976,California,3882244,"Ford, Gerald1976"
...,...,...,...,...,...
445,"Clinton, Hillary",2016,Virginia,1981473,"Clinton, Hillary2016"
446,"Clinton, Hillary",2016,Washington,1742718,"Clinton, Hillary2016"
1627,"Trump, Donald J.",2016,West Virginia,489371,"Trump, Donald J.2016"
1628,"Trump, Donald J.",2016,Wisconsin,1405284,"Trump, Donald J.2016"


In [280]:
#merge dataframed for the state won flag

state_winner_added = final_working_df.merge(find_top_candidates_state_df, how="left", on= ["new", "state"], suffixes=('', 'drop'))

#fill na in flag column for loc and change
state_winner_added["candidatedrop"] = state_winner_added["candidatedrop"].fillna(0)

#loc and change values
state_winner_added.loc[state_winner_added["candidatedrop"] != 0, ["candidatedrop"]] = '1'
state_winner_added.loc[state_winner_added["candidatedrop"] == 0, ["candidatedrop"]] = '0'

#math for % won
state_winner_added["percent_won"] = state_winner_added["candidatevotes"] / state_winner_added["totalvotes"]

#drop and rename columns
state_winner_added = state_winner_added.drop(["yeardrop",  "candidatevotesdrop", "new"], axis=1).rename(columns={"candidatedrop": "state_won_flag"})

final_df = state_winner_added

#final_df = final_df.loc[final_df["candidate"] == "Clinton, Bill"]
#final_df = final_df.loc[final_df["state_won_flag"] == '1']
#final_df = final_df.loc[final_df["year"] == 1992]
final_df

,year,state,candidate,party,totalvotes,Eligible_voting_pop,electoralvotes,candidatevotes,state_won_flag,percent_won
0,1976,Alabama,Other,prohibition,1182850,0.0,9.0,6669,0,0.005638
1,1976,Alabama,"Carter, Jimmy",democrat,1182850,0.0,9.0,659170,1,0.557273
2,1976,Alabama,"Ford, Gerald",republican,1182850,0.0,9.0,504070,0,0.426149
3,1976,Alabama,Other,communist party use,1182850,0.0,9.0,1954,0,0.001652
4,1976,Alabama,Other,libertarian,1182850,0.0,9.0,1481,0,0.001252
...,...,...,...,...,...,...,...,...,...,...
3361,2016,Wyoming,Other,constitution party,258788,429682.0,3.0,2042,0,0.007891
3362,2016,Wyoming,"Clinton, Hillary",democrat,258788,429682.0,3.0,55973,0,0.216289
3363,2016,Wyoming,Other,libertarian,258788,429682.0,3.0,13287,0,0.051343
3364,2016,Wyoming,Other,unknown,258788,429682.0,3.0,278,0,0.001074


In [281]:
#electoral college df
electoral_college = final_df[["year", "state", "candidate","electoralvotes", "state_won_flag"]]
electoral_college = electoral_college.loc[electoral_college["state_won_flag"] == '1']

electoral_college = electoral_college.drop(["state","state_won_flag" ], axis=1)
electoral_college = electoral_college.groupby(["year", "candidate"]).sum().reset_index()

electoral_college["new"] = electoral_college["candidate"] + electoral_college["year"].astype(str)
#electoral_college = electoral_college.sort_values(["year", "electoralvotes"], ascending=False)



electoral_college_win = electoral_college.groupby("year").head(2)
electoral_college_win



,year,candidate,electoralvotes,new
0,1976,"Carter, Jimmy",294.0,"Carter, Jimmy1976"
1,1976,"Ford, Gerald",241.0,"Ford, Gerald1976"
2,1980,"Carter, Jimmy",46.0,"Carter, Jimmy1980"
3,1980,"Reagan, Ronald",489.0,"Reagan, Ronald1980"
4,1984,"Mondale, Walter",10.0,"Mondale, Walter1984"
5,1984,"Reagan, Ronald",525.0,"Reagan, Ronald1984"
6,1988,"Bush, George H.W.",426.0,"Bush, George H.W.1988"
7,1988,"Dukakis, Michael",109.0,"Dukakis, Michael1988"
8,1992,"Bush, George H.W.",168.0,"Bush, George H.W.1992"
9,1992,"Clinton, Bill",367.0,"Clinton, Bill1992"


In [282]:
#start of year level dataset

year_df = final_df.groupby(["year", "candidate"]).sum().reset_index().sort_values(by= ["year", "candidatevotes"], ascending=False).groupby("year").head(1)
popular_vote = year_df[["year", "candidate", "candidatevotes"]]
popular_vote["new"] = popular_vote["candidate"] + popular_vote["year"].astype(str)



year_df = final_df.groupby(["year", "candidate"]).sum().reset_index().sort_values(by= ["year", "candidatevotes"], ascending=False).groupby("year").head(3)

vote = year_df[["year", "candidate", "candidatevotes"]]
vote["new"] = vote["candidate"] + vote["year"].astype(str)

vote = vote.merge(popular_vote, how="left", left_on="new", right_on="new", suffixes=('', 'drop'))
vote = vote.merge(electoral_college, how="left", left_on="new", right_on="new", suffixes=('', 'drop1'))

In [283]:
vote = vote.drop(["new", "yeardrop", "yeardrop1", "candidatevotesdrop", "candidatedrop1"], axis=1)
vote["electoralvotes"] = vote["electoralvotes"].fillna(0)
vote["candidatedrop"] = vote["candidatedrop"].fillna(0)

vote.loc[vote["candidatedrop"] != 0, ["candidatedrop"]] = '1'
#vote.loc[vote["candidatedrop"] == 0, ["candidatedrop"]] = '0'

vote.rename(columns={"candidatedrop": "won_pop_vote_flag"}, inplace=True)

vote

,year,candidate,candidatevotes,won_pop_vote_flag,electoralvotes
0,2016,"Clinton, Hillary",65570751,1,230.0
1,2016,"Trump, Donald J.",62972339,0,305.0
2,2016,Other,6974950,0,0.0
3,2012,"Obama, Barack H.",65632590,1,329.0
4,2012,"Romney, Mitt",59620101,0,206.0
5,2012,Other,3393086,0,0.0
6,2008,"Obama, Barack H.",69252659,1,361.0
7,2008,"McCain, John",59930916,0,174.0
8,2008,Other,1801173,0,0.0
9,2004,"Bush, George W.",62007029,1,286.0


In [284]:
turnout_votes = final_working_df[["year", "totalvotes"]]
turnout_eligible = final_working_df[["year", "Eligible_voting_pop"]]

turnout_votes.drop_duplicates(inplace=True)
#turnout_votes

turnout_eligible.drop_duplicates(inplace=True)
#turnout_eligible

turnout_votes = turnout_votes.groupby("year").sum()
#turnout_votes

turnout_eligible = turnout_eligible.groupby("year").sum().reset_index()

turnout = turnout_votes.merge(turnout_eligible, how = "left", on="year")
turnout

turnout["perc"] = turnout["totalvotes"]/ turnout["Eligible_voting_pop"].round(2)
turnout

,year,totalvotes,Eligible_voting_pop,perc
0,1976,81432514,0.0,inf
1,1980,86322962,157431825.0,0.548320
2,1984,92443573,164946657.0,0.560445
3,1988,91393948,170699171.0,0.535409
4,1992,104372208,176954186.0,0.589826
5,1996,96204092,183628183.0,0.523907
6,2000,105392088,191213067.0,0.551176
7,2004,122121864,199352847.0,0.612592
8,2008,131153400,208197066.0,0.629948
9,2012,128846233,217172515.0,0.593290
